Here is a simple example of how the package can be used and what requirements must be met in order to be able to use the full range of functions.
A basic requirement is the availability of downloaded satellite image data (so far only Sentinel-2 is supported). The data structure should look similar to this: 

![Example data structure](./img/example_data_structure.png)

The most important thing is that the bands in the folder are marked with a "B" and the number and stored as GeoTIFF, e.g. ..._B02.tif, ..._B12.tif. If the data is to be pre-processed for a deep learning model, then a file with the corresponding labels must also be provided for each image and contain the name annotation (see example: annotation.tif).

In [1]:
# Make sure the package is installed
# pip install satellite_datacube

In [3]:
from satellite_datacube.datacube import SatelliteDataCube 
import os 
# Define folder of satellite data and specify some parameters (read more in the doc-string)
satellite = "S2" # or S1 or S12
base_folder = r"D:\SatelliteDataCube\Chimanimani" # path_of_SITS = ".../Sentinel2"
S2_Chimanimani = SatelliteDataCube(satellite=satellite, base_folder=base_folder)


----------------------------------------
Initialized data-cube with following parameter:
- Images from satellite mission: S2
- Base folder: D:\SatelliteDataCube\Chimanimani
- Start-End: 2018-09-06 -> 2019-09-16
- Length of data-cube: 76
----------------------------------------


In [10]:
# Make a selection of images and store them as timeseries 
S2_Chimanimani_ts10 = S2_Chimanimani.create_timeseries(timeseries_length=6)
S2_Chimanimani.save_timeseries(timeseries=S2_Chimanimani_ts10)

Selecting timeseries with 6 satellite images of data-cube


In [11]:
# See selected timeseries and check the length of it
for idx, satellite_image in enumerate(S2_Chimanimani_ts10):
    print(f"{idx+1}: Satellite image on date {satellite_image.date}")

1: Satellite image on date 2018-09-06
2: Satellite image on date 2018-11-20
3: Satellite image on date 2019-01-24
4: Satellite image on date 2019-04-29
5: Satellite image on date 2019-07-03
6: Satellite image on date 2019-09-11


In [12]:
# Create patches for satellite image, masks and the global mask of selected timeseries with patch size of 256px
S2_Chimanimani_pt256_ts10 = S2_Chimanimani.create_patches(patch_size=256, selected_timeseries=S2_Chimanimani_ts10)

Generating patches of size 256x256px for 6 satellite images with corresponding masks and for one global mask:
-> Start with satellite images of date 2018-09-06
-> Start with satellite images of date 2018-11-20
-> Start with satellite images of date 2019-01-24
-> Start with satellite images of date 2019-04-29
-> Start with satellite images of date 2019-07-03
-> Start with satellite images of date 2019-09-11
-> Start with satellite masks of date 2018-09-06
-> Start with satellite masks of date 2018-11-20
-> Start with satellite masks of date 2019-01-24
-> Start with satellite masks of date 2019-04-29
-> Start with satellite masks of date 2019-07-03
-> Start with satellite masks of date 2019-09-11
-> Start with global_mask


In [13]:
for source, array in S2_Chimanimani_pt256_ts10.items():
    print(f"Patches of {source.upper()} with shape {array.shape} are stored in dict with key: {source}")
    if source == "global_mask":
        print(f"There are {array.shape[0]} patches with {array.shape[1]} band and a resolution of {array.shape[-1]}px")
    else:
        print(f"There are {array.shape[0]} patches with {array.shape[1]} timesteps and {array.shape[2]} band(s) and a resolution of {array.shape[-1]}px")

Patches of IMAGES with shape (448, 6, 10, 256, 256) are stored in dict with key: images
There are 448 patches with 6 timesteps and 10 band(s) and a resolution of 256px
Patches of MASKS with shape (448, 6, 1, 256, 256) are stored in dict with key: masks
There are 448 patches with 6 timesteps and 1 band(s) and a resolution of 256px
Patches of GLOBAL_MASK with shape (448, 1, 256, 256) are stored in dict with key: global_mask
There are 448 patches with 1 band and a resolution of 256px


In [14]:
S2_Chimanimani_pt256_ts10 = S2_Chimanimani.filter_patches(
    patches=S2_Chimanimani_pt256_ts10,
    class_values=[1],
    class_ratio=(80,20)
    )

In [15]:
for source, array in S2_Chimanimani_pt256_ts10.items():
    print(f"Patches of {source.upper()} with shape {array.shape} are stored in dict with key: {source}")
    if source == "global_mask":
        print(f"There are {array.shape[0]} patches with {array.shape[1]} band and a resolution of {array.shape[-1]}px")
    else:
        print(f"There are {array.shape[0]} patches with {array.shape[1]} timesteps and {array.shape[2]} band(s) and a resolution of {array.shape[-1]}px")

Patches of IMAGES with shape (60, 6, 10, 256, 256) are stored in dict with key: images
There are 60 patches with 6 timesteps and 10 band(s) and a resolution of 256px
Patches of MASKS with shape (60, 6, 1, 256, 256) are stored in dict with key: masks
There are 60 patches with 6 timesteps and 1 band(s) and a resolution of 256px
Patches of GLOBAL_MASK with shape (60, 1, 256, 256) are stored in dict with key: global_mask
There are 60 patches with 1 band and a resolution of 256px


In [17]:
# Save filtered patches 
patches_folder = r"D:\SatelliteDataCube\Chimanimani\patches"
S2_Chimanimani.save_patches(patches=S2_Chimanimani_pt256_ts10, patches_folder=patches_folder)

256 6
Saved patches from source images as array with shape: (60, 6, 10, 256, 256)
Saved patches from source masks as array with shape: (60, 6, 1, 256, 256)
Saved patches from source global_mask as array with shape: (60, 1, 256, 256)


In [20]:
from satellite_datacube.utils import sanity_check
sanity_check(patches=S2_Chimanimani_pt256_ts10)

TypeError: sanity_check() missing 1 required positional argument: 'self'